In [1]:
import os
import hashlib
import pandas as pd
import numpy as np

csv_path = os.path.join(os.curdir, "datasets", "dataset - Privacy-Engineering.csv")
df = pd.read_csv(csv_path)

In [2]:
columns_to_drop = ["ancestry"]
df = df.drop(columns_to_drop, axis=1)

In [3]:
salt = b"\x91\xff=s\x18?JH\x940\xf1$wN\xa4\xbf\x9f\xcdi\x8c\xa2\xf3\xe3I\xeb\xf4\x1f\xc0y\x94\x84\xa8"

df.id = df.id.apply(lambda x : hashlib.sha512(str(x).encode() + salt).hexdigest())
df.dob = df.dob.apply(lambda x : x.split("/")[1])

In [4]:
df.head(5)

,id,gender,dob,zipcode,education,employment,children,marital_status,number_vehicles,commute_time,accommodation,disease
0,97eb7a57528a1baea5a7be646149f9d75cc53f3c5ba87f...,female,1950,26904,High School,Retired,2,married,2,0.0,Own house,Alzheimer's disease
1,730151ea90e546b7b8fe5911d91ac943a9095ebb6691ba...,female,1940,78088,PhD/md,Retired,1,married,1,0.0,Rent flat,multiple sclerosis
2,ef150a1bf99c1188ff526b1408d959e2cdb1729c4f1153...,female,1966,51502,Bachelor,Employed,1,married,0,0.1,Rent flat,heart disease
3,d9240d1730615fa39e50b22de35439c79814e162454e9e...,female,1943,54080,Bachelor,Retired,1,married,0,0.0,Rent room,endometriosis
4,a445d7cdb7efbbc10c59a27331869c8151ddb55313fcd0...,female,1947,68785,High School,Retired,1,married,0,0.0,Rent flat,gastritis


In [5]:
zip_path = os.path.join(os.curdir, "datasets", "uszips.csv")
df_zip = pd.read_csv(zip_path, dtype={"zip": 'string'})
df_zip = pd.DataFrame(df_zip)

In [6]:
df_zip.head()

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,county_fips,county_name,county_weights,county_names_all,county_fips_all,imprecise,military,timezone
0,00601,18.18004,-66.75218,Adjuntas,PR,Puerto Rico,True,NaN,17242,111.4,72001,Adjuntas,"{'72001':99.43,'72141':0.57}",Adjuntas|Utuado,72001|72141,False,False,America/Puerto_Rico
1,00602,18.36073,-67.17517,Aguada,PR,Puerto Rico,True,NaN,38442,523.5,72003,Aguada,{'72003':100},Aguada,72003,False,False,America/Puerto_Rico
2,00603,18.45439,-67.12202,Aguadilla,PR,Puerto Rico,True,NaN,48814,667.9,72005,Aguadilla,{'72005':100},Aguadilla,72005,False,False,America/Puerto_Rico
3,00606,18.16724,-66.93828,Maricao,PR,Puerto Rico,True,NaN,6437,60.4,72093,Maricao,"{'72093':94.88,'72121':1.35,'72153':3.78}",Maricao|Yauco|Sabana Grande,72093|72153|72121,False,False,America/Puerto_Rico
4,00610,18.29032,-67.12243,Anasco,PR,Puerto Rico,True,NaN,27073,312.0,72011,Añasco,"{'72003':0.55,'72011':99.45}",Añasco|Aguada,72011|72003,False,False,America/Puerto_Rico


In [7]:
zip_match = {}
for i, row in df_zip.iterrows():
    row_zip = str(row['zip'])[0:2]
    row_state_id = row['state_id']
    row_state_name = row['state_name']

    if row_zip not in zip_match: zip_match[row_zip] = f"({row_state_id}) {row_state_name}"


In [8]:
def find_state(zip_code):
    res = zip_match.get(str(zip_code)[0:2])
    if res is None: return "Not found"
    return res

df['state'] = df.zipcode.apply(find_state)
df = df.drop("zipcode", axis=1)

In [9]:
df.head()

,id,gender,dob,education,employment,children,marital_status,number_vehicles,commute_time,accommodation,disease,state
0,97eb7a57528a1baea5a7be646149f9d75cc53f3c5ba87f...,female,1950,High School,Retired,2,married,2,0.0,Own house,Alzheimer's disease,(WV) West Virginia
1,730151ea90e546b7b8fe5911d91ac943a9095ebb6691ba...,female,1940,PhD/md,Retired,1,married,1,0.0,Rent flat,multiple sclerosis,(TX) Texas
2,ef150a1bf99c1188ff526b1408d959e2cdb1729c4f1153...,female,1966,Bachelor,Employed,1,married,0,0.1,Rent flat,heart disease,(IA) Iowa
3,d9240d1730615fa39e50b22de35439c79814e162454e9e...,female,1943,Bachelor,Retired,1,married,0,0.0,Rent room,endometriosis,(WI) Wisconsin
4,a445d7cdb7efbbc10c59a27331869c8151ddb55313fcd0...,female,1947,High School,Retired,1,married,0,0.0,Rent flat,gastritis,(NE) Nebraska


In [10]:
def more_than_2_children(x):
    if isinstance(x, str): return x
    elif x == 0: return '0'
    elif x == 1: return '1'
    elif x > 1: return '2+'

def more_than_1_hour(x):
    if isinstance(x, str): return x
    elif x == 0.0: return '0'
    elif x <= 1.0: return '1.0-'
    elif x > 1.0: return '+1.0'

df.children=df.children.apply(more_than_2_children)
df.commute_time=df.commute_time.apply(more_than_1_hour)
df.head()

IndexError: list index out of range